In [3]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import re

#df 생성
import pandas as pd
import numpy as np

driver = Chrome('./chromedriver')
url = 'https://comic.naver.com/webtoon/creation'
driver.get(url)


/var/folders/b2/xf80p57564z4z_vr5vzw0nd80000gn/T/ipykernel_29129/1694458432.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = Chrome('./chromedriver')


In [6]:
df = pd.read_csv("../../csv/after_pre/drop_webtoon_preprocessing_221130.csv",encoding="cp949",index_col=0)
df

,title,status,author,intro,likes,episodes,first_register_date,last_register_date,age,rate,genre1_pre,genre2_pre
1,가난을 등에 업은 소녀,1,B급달궁 / 오은지,흔하디 흔한 재벌후계자와 캔디도 울고 갈 박복한 가난소녀의 파란만장 동거기!인기작 ...,344,28.0,2020.10.19,2020.10.19,0,9.6,0,0
2,가담항설,1,랑또,이번 주인공은 돌이다!돌이지만 동료도 모으고 악당도 물리친다!랑또 작가표 동양 판타...,100000,247.0,2016.06.01,2016.08.03,1,9.9,0,2
3,가령의 정체불명 이야기,1,가령,어느 날... 인어가 내게 말을 걸어왔다. 눈을 떠보니 총구가 있었다. 그리고 몸이...,38392,25.0,2019.08.19,2019.09.02,2,9.9,2,1
4,가비지타임,0,2사장,"한국 최초 한국형 고교스포츠 웹툰!열혈따윈 개나 줘, 낙오자들 뿐인 농구부의 운명은?",81510,149.0,2022.05.07,2022.11.19,0,9.9,0,9
5,가상&RPG,0,주다현,"""사람은 좀 죽여도 싸우는건 싫어해요."" 게임에서조차 집 바깥을 나가지 않는 '제리...",357,42.0,2022.08.17,2022.10.19,1,9.9,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
2040,7FATES: CHAKHO,0,HYBE,"타락한 도시 신시, 길에서 마주친 수수께끼의 사내가 제하에게 의문의 말을 던진다.하...",746,38.0,2022.09.16,2022.11.18,1,9.3,0,2
2041,7아이언,0,태발 / 김홍태,"사고로 오른손을 잃은 골프천재 주아이, 절망하는 시간조차 아깝다!아이는 프로가 되기...",52,73.0,2021.11.11,2021.11.11,1,9.7,0,1
2042,8월의 눈보라,1,김리아,교복모델이었던 보라는 암흑 고3 시절을 거치며 이전의 모습은 온데간데 없다.설상가상...,96439,102.0,2018.11.03,2019.01.05,0,8.9,0,0
2043,99강화나무몽둥이,0,홍실 / 지페리,약육강식이 진리인 가상 현실 게임 '크로노 라이프'.허접이라며 무시만 당하던 '나'...,1889,41.0,2022.09.16,2022.11.18,1,9.7,0,2


In [7]:
keyword = ['로맨스', '완결로맨스', '소년왕도물', '판타지', '완결판타지', '최강자전', '2018 최강자전', '드라마', '완결드라마', '2017 최강자전', '농구', '스포츠성장', '스포츠', '2019 최강자전', '게임판타지', '개그', '완결무료', '완결개그', '드라마&영화 원작웹툰', '일상', '완결일상', '2020 지상최대공모전', '계약연애', '연예계', '삼각관계', '소꿉친구', '2021 지상최대공모전', '학원로맨스', '친구>연인', '사이비종교', '스릴러', '완결스릴러', '캠퍼스로맨스', '학원물', '액션', '소설원작', '오피스로맨스', '사내연애', '성인웹툰', '병맛', '음식요리', '2019 지상최대공모전', '이능력배틀물', '먼치킨', '타임슬립', '완결액션', '하이틴', '동물', '감성드라마', '힐링', '성장드라마', '유료화임박', '프리퀄', '아포칼립스', '느와르', '서스펜스', '머니게임', '비밀연애', '전남친', '감성', '완결감성', '환골탈태', '격투기', '로맨스코미디', '청춘로맨스', '사이다', '서바이벌', '복수극', '직진남', '인외존재', '이세계', '구원서사', '인플루언서', '세계관', '회귀', '판무', '무협/사극', '퓨전사극', '육아물', '재회', '역하렘', '까칠남', '고자극로맨스', '뱀파이어', '직업드라마', '밀리터리', '완결스포츠', '계략여주', '2030연애', '오컬트', '로판', '빙의', '현실로맨스', '혐관로맨스', '레트로', '공감성수치', '2016 최강자전', '걸크러시', '동양풍판타지', '2020 최강자전', 'sf', '감염', '괴담', '연상연하', '다크히어로', '두뇌싸움', '하이퍼리얼리즘', '레드아이스 스튜디오', '헌터물', '짝사랑', '다크판타지', '선결혼후연애', '의학드라마', '성별반전', '궁중로맨스', '슈퍼스트링', '좀비', '레드스트링', '법정드라마', '2013 최강자전', '야구', '아이돌연애', '완결무협/사극', '차원이동', '힘숨찐', '2015 최강자전', '역사물', '2014 최강자전', '히어로', '축구', '스핀오프', '2021 최강자전', '블루스트링', '', '영화원작웹툰']

In [12]:
print(keyword)

['로맨스', '완결로맨스', '소년왕도물', '판타지', '완결판타지', '최강자전', '2018 최강자전', '드라마', '완결드라마', '2017 최강자전', '농구', '스포츠성장', '스포츠', '2019 최강자전', '게임판타지', '개그', '완결무료', '완결개그', '드라마&영화 원작웹툰', '일상', '완결일상', '2020 지상최대공모전', '계약연애', '연예계', '삼각관계', '소꿉친구', '2021 지상최대공모전', '학원로맨스', '친구>연인', '사이비종교', '스릴러', '완결스릴러', '캠퍼스로맨스', '학원물', '액션', '소설원작', '오피스로맨스', '사내연애', '성인웹툰', '병맛', '음식요리', '2019 지상최대공모전', '이능력배틀물', '먼치킨', '타임슬립', '완결액션', '하이틴', '동물', '감성드라마', '힐링', '성장드라마', '유료화임박', '프리퀄', '아포칼립스', '느와르', '서스펜스', '머니게임', '비밀연애', '전남친', '감성', '완결감성', '환골탈태', '격투기', '로맨스코미디', '청춘로맨스', '사이다', '서바이벌', '복수극', '직진남', '인외존재', '이세계', '구원서사', '인플루언서', '세계관', '회귀', '판무', '무협/사극', '퓨전사극', '육아물', '재회', '역하렘', '까칠남', '고자극로맨스', '뱀파이어', '직업드라마', '밀리터리', '완결스포츠', '계략여주', '2030연애', '오컬트', '로판', '빙의', '현실로맨스', '혐관로맨스', '레트로', '공감성수치', '2016 최강자전', '걸크러시', '동양풍판타지', '2020 최강자전', 'sf', '감염', '괴담', '연상연하', '다크히어로', '두뇌싸움', '하이퍼리얼리즘', '레드아이스 스튜디오', '헌터물', '짝사랑', '다크판타지', '선결혼후연애', '의학드라마', '성별반전', '궁중로맨스', '슈퍼스트링', '좀비', '레드스트링', '법정드라

장르 : 하이퍼리얼리즘, 헌터물 , 다크판타지 , 의학드라마 , 궁중로맨스, 법정드라마, 완결무협/사극, 역사물, 히어로, 스핀오프
분위기 : 
수상작 : 2013 최강자전, 2015 최강자전, 2014 최강자전, 2021 최강자전
소재(주제) : 감염 , 괴담 , 두뇌싸움 , 성별반전, 좀비, 야구, 아이돌연애, 차원이동, 힘숨찐, 축구
등장인물관계 : 연상연하, 다크히어로 , 짝사랑 ,선결혼후연애
원작웹툰 : 영화원작웹툰

레드아이스 스튜디오, 슈퍼스트링, 레드스트링, 블루스트링 

In [ ]:
장르 = ["액션","성인웹툰","이능력배틀물",'프리퀄','느와르','로맨스코미디','이세계','판무','무협/사극','퓨전사극','로판','동양풍판타지','sf','역하렘','로맨스','판타지','드라마','개그','일상','스릴러','소년왕도물','스포츠','하이퍼리얼리즘','헌터물','다크판타지','역사물','히어로','스핀오프']
소재 = ['뱀파이어','타임슬립','먼치킨','하이틴','아포칼립스','서바이벌','궁중로맨스','고자극로맨스','현실로맨스','캠퍼스로맨스','혐관로맨스','학원로맨스','청춘로맨스','오피스로맨스','사내연애','음식요리','동물','성장드라마','머니게임','비밀연애','환골탈태','격투기','복수극','회귀','직업드라마','법정드라마','인외존재','인플루언서','세계관','육아물','밀리터리','2030연애','오컬트','빙의','게임판타지','농구','스포츠성장','연예계','사이비종교','의학드라마','소재','두뇌싸움','성별반전','좀비','야구','아이돌연애','차원이동','힘숨찐','축구','다크히어로']
분위기 = ['서스펜스','병맛','감성드라마','힐링','감성','사이다','레트로','공감성수치','괴담']
수상작 = ['2019 지상최대공모전','2016 최강자전','2020 최강자전','최강자전','2018 최강자전','2019 최강자전','2020 지상최대공모전','2021 지상최대 공모전','2013 최강자전','2015 최강자전', '2014 최강자전', '2021 최강자전']
등장인물/관계 = ['전남친','재회','직진남','구원서사','까칠남','계략여주','걸크러시','계약연애','삼각관계','소꿉친구','친구>연인','연상연하','짝사랑','선결혼후연애']
원작웹툰 = ['소설원작','드라마&영화 원작웹툰','영화원작웹툰']

뺄거 = 완결액션,유료화임박,완결감성,완결스포츠,완결로맨스,완결판타지,완결드라마,완결개그,완결일상,완결스릴러,완결무료,완결무협/사극,레드아이스 스튜디오, 슈퍼스트링,레드스트링,블루스트링

In [10]:
del_key = ['완결액션','유료화임박','완결감성','완결스포츠','완결로맨스','완결판타지','완결드라마','완결개그','완결일상','완결스릴러','완결무료','완결무협/사극','레드아이스 스튜디오', '슈퍼스트링','레드스트링','블루스트링']

In [13]:
len(del_key)

16

In [18]:
n=0
for i in keyword:
    if i in del_key:
        print(i)

1
완결로맨스
2
완결판타지
3
완결드라마
4
완결무료
5
완결개그
6
완결일상
7
완결스릴러
8
완결액션
9
유료화임박
10
완결감성
11
완결스포츠
12
레드아이스 스튜디오
13
슈퍼스트링
14
레드스트링
15
완결무협/사극
16
블루스트링
